# 957. Prison Cells After N Days


## Topic Alignment
- MLE Connection: Modeling finite-state systems with cycle detection supports feature evolution simulations.
- Hash Table Role: Store seen cell states keyed by bitmask to detect loops and jump ahead.
- Interview Angle: Demonstrates hashing immutable state representations and leveraging modular arithmetic to skip redundant work.


## Metadata Summary
- Source: https://leetcode.com/problems/prison-cells-after-n-days/
- Tags: Array, Hash Table, Simulation
- Difficulty: Medium
- Recommended Review Priority: Medium


## Problem Statement
There are 8 prison cells in a row, each either occupied (1) or vacant (0). Each day, cells change state according to the following rule: if a cell has two adjacent neighbors in the same state, it becomes occupied; otherwise it becomes vacant. Both ends have a single neighbor outside the array and are always treated as vacant. Given an array cells representing the initial state and an integer n, return the state of the prison after n days.


## Progressive Hints
- Hint 1: Each day depends only on the previous state; there are at most 2^6 distinct interior patterns.
- Hint 2: Encode the interior cells as a bitmask so you can store previously seen states in a hash map.
- Hint 3: Once a state repeats, you have found a cycle and can jump ahead by taking n modulo the cycle length.


## Solution Overview
Simulate the transition function while caching each encountered state in a hash map from bitmask to day index. When a repeat is detected, compute the remaining days modulo the cycle length and fast-forward to the final state without iterating the entire n days.


## Detailed Explanation
Because there are only 8 cells and the two edges always become 0 after the first day, the system has at most 2^6 unique states. We can encode the state as an integer bitmask (using only the interior cells) or as a tuple and store it in a dictionary mapping to the day index when it first appears.

Algorithm steps:
1. Convert the current cell configuration into a tuple to serve as a hashable key.
2. Iterate day by day. For each day, compute the next cell state by checking neighbors.
3. If the newly produced state is already in the dictionary, a cycle of length `cycle = current_day - first_seen_day` is found.
4. Reduce the remaining steps `remaining = (n - current_day) % cycle` and replay the transition that many times to reach the final state.
5. Return the state after all simulated days.

This strategy turns a potentially billion-step simulation into at most a few dozen iterations thanks to cycle detection.


## Complexity Trade-off Table
| Approach | Time Complexity | Space Complexity |
| --- | --- | --- |
| Straight simulation for n days | O(n) | O(1) |
| Cycle detection with hash map | O(states) <= O(2^6) | O(states) |


## Reference Implementation


In [ ]:
from typing import List, Tuple


class Solution:
    def prisonAfterNDays(self, cells: List[int], n: int) -> List[int]:
        seen: dict[Tuple[int, ...], int] = {}
        state = tuple(cells)
        day = 0

        while day < n:
            if state in seen:
                cycle_length = day - seen[state]
                remaining = (n - day) % cycle_length
                for _ in range(remaining):
                    state = self._next_state(state)
                return list(state)

            seen[state] = day
            state = self._next_state(state)
            day += 1

        return list(state)

    def _next_state(self, state: Tuple[int, ...]) -> Tuple[int, ...]:
        next_cells = [0] * len(state)
        for i in range(1, len(state) - 1):
            next_cells[i] = int(state[i - 1] == state[i + 1])
        return tuple(next_cells)


## Complexity Analysis
- Time Complexity: O(min(n, S)), where S is the number of distinct states (<= 64). Once a cycle is detected the remaining steps are reduced modulo S.
- Space Complexity: O(S) to store the first occurrence of each state.
- Bottlenecks: Hashing small tuples and iterating the cycle remainder dominate but are negligible.


## Edge Cases & Pitfalls
- Handle n so large that the cycle detection logic must trigger; skipping the modulo step causes timeouts.
- Ensure edges are always zero after the first transition; do not read out-of-bounds neighbors.
- Use an immutable structure (tuple) as the hash key to avoid accidental mutation between iterations.


## Follow-up Variants
- Generalize to k cells or multi-dimensional grids and analyze state-space growth.
- Track and return the cycle length explicitly in addition to the final state.
- Introduce stochastic transitions and estimate convergence using Monte Carlo simulation.


## Takeaways
- Small state spaces invite cycle detection to replace long simulations.
- Hashing normalized representations is a powerful way to memoize visited configurations.
- Modulo arithmetic converts repeated patterns into constant-time skips.


## Similar Problems
| Problem ID | Problem Title | Technique |
| --- | --- | --- |
| 202 | Happy Number | Cycle detection |
| 142 | Linked List Cycle II | Detecting cycles |
| 266 | Palindrome Permutation | State hashing |
